In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import random


data = pd.read_csv('/Users/aoamacsplace/Documents/Projects/Online retail/data-models/simulated_data.csv')   

In [7]:
def distribute_tokens(data, min_tokens, max_tokens):
    total_tokens = sum(data['op_token'])
    total_tokens_voting = sum(data['op_tokens_voting'])
    total_reputation = sum(data['reputation_score'])
    total_op_tx_nft = sum(data['op_tx_nft'])
    
    target_tokens = len(data) * (min_tokens + max_tokens) / 2
    target_tokens_voting = len(data) * (min_tokens + max_tokens) / 2
    target_reputation = len(data) * (min_tokens + max_tokens) / 2
    target_op_tx_nft = len(data) * (min_tokens + max_tokens) / 2
    
    scaling_factor = target_tokens / total_tokens
    scaling_factor_voting = target_tokens_voting / total_tokens_voting
    scaling_factor_reputation = target_reputation / total_reputation
    scaling_factor_op_tx_nft = target_op_tx_nft / total_op_tx_nft
    
    distributed_tokens = {}
    remaining_tokens = target_tokens
    
    for i in range(len(data)):
        ensname = data['ensName'][i]
        op_token = data['op_token'][i]
        op_tokens_voting = data['op_tokens_voting'][i]
        reputation_score = data['reputation_score'][i]
        op_tx_nft = data['op_tx_nft'][i]
        
        allocated_tokens = round(op_token * scaling_factor)
        allocated_tokens_voting = round(op_tokens_voting * scaling_factor_voting)
        allocated_reputation = round(reputation_score * scaling_factor_reputation)
        allocated_op_tx_nft = round(op_tx_nft * scaling_factor_op_tx_nft)
        
        allocated_tokens = min(max_tokens, max(min_tokens, allocated_tokens))
        allocated_tokens_voting = min(max_tokens, max(min_tokens, allocated_tokens_voting))
        allocated_reputation = min(max_tokens, max(min_tokens, allocated_reputation))
        allocated_op_tx_nft = min(max_tokens, max(min_tokens, allocated_op_tx_nft))
        
        total_allocated = allocated_tokens + allocated_tokens_voting + allocated_reputation + allocated_op_tx_nft
        
        distributed_tokens[ensname] = total_allocated
        remaining_tokens -= total_allocated
    
    # Distribute remaining tokens randomly
    while remaining_tokens > 0:
        ensname = random.choice(list(data['ensName']))
        allocated_tokens = min(max_tokens - distributed_tokens[ensname], remaining_tokens)
        distributed_tokens[ensname] += allocated_tokens
        remaining_tokens -= allocated_tokens
    
    return distributed_tokens

In [8]:
min_tokens = 100
max_tokens = 1000


distributed_tokens = distribute_tokens(data, min_tokens, max_tokens)
df = pd.DataFrame({'ensName': list(distributed_tokens.keys()), 'DistributedTokens': list(distributed_tokens.values())})
print(df)

        ensName  DistributedTokens
0           NaN               2529
1      a68cdb55               2831
2      26474088               1942
3      db462029               2267
4      16b7fdbc               3339
...         ...                ...
19994  a5c849eb               2193
19995  b72ad97c               1627
19996  ad24df0e               2661
19997  6996de1e               2558
19998  16602839               2495

[19999 rows x 2 columns]


In [13]:

df_sorted = df.sample(10)

fig = go.Figure(data=[go.Bar(x=df_sorted['ensName'], y=df_sorted['DistributedTokens'])])

fig.update_layout(
    title='Top 10 ENS Names by Distributed Tokens',
    xaxis=dict(title='ENS Name'),
    yaxis=dict(title='Distributed Tokens'),
    xaxis_tickangle=-45,
    autosize=False,
    width=800,
    height=500
)

fig.show()

In [14]:

data.columns

Index(['walletaddress', 'date_created', 'op_token', 'op_tokens_voting',
       'ensName', 'ensDescription', 'ensUrl', 'ensTelegram', 'ensTwitter',
       'ensGithub', 'ensEmail', 'followCount', 'followCountSpace',
       'followCountNonSpace', 'totalVotes', 'totalProposalInteraction',
       'totalProposalInteractionInSpace', 'totalProposalInteractionNonSpace',
       'totalNft', 'rewardsNft', 'reputation_score', 'op_tx', 'purchased_nft',
       'op_tx_nft'],
      dtype='object')